# PRÁCTICA 1 MINERIA DE DATOS

*Autores: Marina Ceballos Verdejo y Jesús Martínez Garrido*

# 1. Diabetes database

Primero importamos todo lo necesario:

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

# Local application
import utilidad as utils

Fijamos la semilla para que los datos sean reproducibles

In [ ]:
seed = 27912

## 1.1 Acceso y almacenamiento de datos

Cargamos la base de datos:

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index = None
target = "Outcome"

data = utils.load_data(filepath, index, target)

Obtenemos los 5 primeros ejemplos para visualizar los datos.

In [ ]:
data.head(5)

De aquí podemos obtener la siguiente información:

* Pregnancies es un número entero
* Glucosa es un número entero que se mueve en el rango [80, 200] más o menos.
* BloodPressure es otro entero y podría estar en el rango[40,80]
* SkinThickness es también un número entero con números que podrían estar en el rango [0, 40]
* Insulin es otro entero con números que podrían estar entre [0, 180]
* BMI es un número real con datos que podrían estar comprendidos entre [20.0, 50.0]
* DiabetesPedigreeFunction es una variable con datos reales del rango [0, 1] pero puede haber algún dato anómalo como pasa en el ejemplo 4
* Age es un número entero con datos comprendidos entre [0, 100]
* Outcome es la variable objetivo con únicos valores de 0 y 1

Obviamente no podemos diseñar unos rangos con los primeros 5 ejemplos ya que no es una muestra muy representativa de la población. Es un pequeño estudio que podemos sacar de los primeros 5 ejemplos para ir familiarizándonos con los datos. Este estudio se llevará en profundidad en el apartado "Análisis exploratorio de datos"

A continuación, vamos a obtener otros 5 ejemplos aleatorios para poder obtener más información

In [ ]:
data.sample(5, random_state=seed)

Aquí podemos ver datos que pueden variar con respecto a los cinco primeros.
* En la variable BloodPressure podemos ver que pueden existir datos más altos (84 y 110)
* Insulin también tiene un dato que se aleja del intervalo definido anteriormente (240)

Ahora procedemos a dividir los datos:

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

Volvemos a obtener los ejemplos aleatorios para comprobar que se ha realizado correctamente la división

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Dividimos los datos en train/test para tener una parte de los datos para entrenar y otra para llevar a cabo la evaluación de nuestro modelo. En nuestro caso hemos puesto que el 75% de los datos serán train y el 25% test:

In [ ]:
train_size = 0.75

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Obtenemos un subconjunto de cada:

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
X_test.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)

In [ ]:
y_test.sample(5, random_state=seed)

Unimos las variables predictoras con la variable clase en ambos conjuntos de datos:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

In [ ]:
data_test = utils.join_dataset(X_test, y_test)

In [ ]:
data_train.sample(5, random_state=seed)

In [ ]:
data_test.sample(5, random_state=seed)

## 1.2. Análisis exploratorio de datos

Antes de comenzar el preprocesamiento vamos a observar las características del conjunto de datos (de esta forma, haremos el preprocesamiento de una manera más efectiva).

### Descripción del conjunto de datos

In [ ]:
data_train.shape

Tal y como se puede observar, el conjunto de datos de entrenamiento está formado por 576 casos y 9 variables (8 variables predictoras y 1 variable clase).

In [ ]:
data_train.info(memory_usage=False)

Nuestras variables predictoras son continuas (todas ellas enteros menos BMI y DiabetesPedigreeFunction que son decimales). No hay ningún dato perdido porque todos tienen 576 non-null (número de ejemplos).

In [ ]:
y_train.cat.categories

Esto es, nuestra variable clase es multivariada con dos estados.

### Visualización de las variables

In [ ]:
utils.plot_histogram(data_train)

Los histogramas de algunas variables predictoras representan una distribución normal (BMI, BloodPressure, Glucose), aunque las tres presentan algún dato ruidoso ya que son valores erróneos, por ejemplo:
* BMI, en 0. En 80 se puede dar el caso pero es muy poco probable.
* BloodPressure, en 0.
* Glucose, en 0.

Los datos suelen estar entre los siguientes rangos:
* BMI se encuentra en el rango [17,69]
* BloodPressure se encuentra en el rango [20, 124]
* Glucose se encuentra en el rango [55,200]

También podemos encontrar algunas distribuciones más complejas, por ejemplo en la variable Skin Thickness encontramos lo que podrían ser dos distribuciones normales juntas en el rango [5,64] (mixtura de distribuciones normales). 
Además, encontramos un dato anómalo en el rango [95,99], ya que se aleja de la distribución de la variable. Tiene valores ruidosos ya que es imposible tener 0 de Skin Thickness.


Por otro lado, las distribuciones de las variables Pregnancies, DiabetesPedigreeFunction y Age son también algo más complejas que una normal ya que podría ser una distribución normal pero en este caso es asimétrica por la parte derecha.
* La distribución de la variable Pregancies se encuentra en el rango [0,17].
* La distribución de la variable DiabatesPedrigreeFunction se encuentra en el rango [0,2.5].
* La distribución de la variable Age se encuentra en el rango [20,71], y encontramos un dato anómalo en el valor 80 porque se aleja de la distribución de la variable.

Finalmente, en la variable Insuline encontramos una distribución normal en el rango [20,700], aunque podemos apreciar una mixtura de esta distribución con otra uniforme que se encuentra al principio con valores entre [0,19]. 
A partir del valor 700 encontramos unos valores anómalos hasta el valor 800, ya que esán alejados de la distribución de la variable.


Continuamos visualizando las variables categóricas del problema:

In [ ]:
utils.plot_barplot(data_train)

Lo que podemos observar es que las dos clases de la variable objetivo del problema no tienen el mismo número de casos, por lo tanto no es una muestra balanceada, ya que hay más personas que no tengan diabetes. Es algo que suele pasar en los estudios médicos, el porcentaje de la población afectada suele ser menor.


A continuación, vamos a proceder a realizar un análisis multivariado:

In [ ]:
utils.plot_pairplot(data_train, target="Outcome")

Viendo este análisis multivariado, primero analizamos las variables predictoras por separado:
* En Pregnancies, vemos como los valores están bastante dispersos, pero hay una diferencia notable entre los puntos rojos y los azules, ya que los azules tienen valores de Pregnancies más altos que los rojos, por lo que los encontramos más a la derecha. También, como sabemos por el histograma los ejemplos de pregnancies altos son pocos y los valores de clase '1' se mantienen en las gráficas de Pregnancies, podemos decir que Pregnancies y Outcome están relacionadas.
* En Glucose, ocurre lo mismo que en el caso anterior.
* En BloodPressure, los datos están más mezclados que en las variables anteriores viendo como los puntos azules están más levemente desplazados hacia la derecha (valores más altos) que los rojos.
* SkinThickness, insulin, BMI y DiabetesPedigreeFunction, ocurre lo mismo que en el caso anterior pero podemos notar como en el valor 0 predominan los puntos rojos.
* En la variable predictora Age la diferencia es menos notable porque los puntos están más superpuestos los unos de los otros. Pero al ver en el histograma de 'edad' que existen pocos ejemplos de personas mayores y los puntos azules se mantienen podemos decir que la edad está relacionada con la variable clase.


Ahora procedemos a analizar variables predictoras de forma conjunta:
* Vemos que en la gráfica de Insulin x Pregnancies los valores de clase 0 predominan con valores de insulin y pregnancies bajos y los valores de clase 1 predominan con insulin y pregnancies algo más altos
* En las gráfica DiabetesPedigreeFunction x Pregnancies pasa lo mismo que en el caso anterior
* En la gráfica Glucose x Insulin podemos ver como cuando la glucosa sube la insulina también lo hace (es decir, existe una relación directa entre ambas variables).
* En la gráfica de Insulin x Age se puede ver una relación inversa entre ambas variables predictoras ya que cuando aumenta la edad, la variable insulina es más baja.


In [ ]:
utils.plot_heatmap(data_train)

En esta gráfica podemos encontrar varias relaciones claras:

* Age y Pregnancies: relación directa
* Insulin y Glucose: hay una relación directa entre ambas variables
* Insulin y SkinThickness: tienen otra relación directa
* BMI y SkinThickness: relación directa

También debemos de destacar que no hay relaciones inversas (o son poco significantes).

In [ ]:
utils.plot_box(data_train)

Con estos diagramas de cajas podemos sacar rangos más precisos sobre cada variable (sin tener en cuenta los valores anómalos representados por puntos en el diagrama). Obtendremos intervalos mas exáctos que en el histograma (ya que en este tipo de gráficos aparecen valores máximos, mínimos y anómalos):
* Pregnancies [0, 13] tiene dos valores anómalos en 14 y 17
* Glucose [56, 199] tiene un valor anómalo en 0 (en este caso es un valor ruidoso porque no es posible tener la glucosa en 0)
* BloodPressure [38, 106] con 6 valores anómalos en 24, 30, 108, 110, 114, 122 y un valor ruidoso en 0 ya que no se puede tener la presión de sangre como 0.
* SkinThickness [0,63] con un valor anómalo en 99. Además, los valores que se encuentran 0 (o cerca del 0), aunque sean muchos casos, son ruidosos ya que no tiene sentido tener ese grosor de piel.
* Insulin [0, 325] en este caso tener 0 de insulina es algo usual en los diabéticos, por lo que no es un valor ruidoso. Hay bastantes valores anómalos, siendo los más significantes 846, 744, 680 (estos son valores demasiado altos de insulina por lo que podrían ser considerados ruidosos).
* BMI [18.2, 50] con valores anómalos en 52.3, 52.9, 53.2, 57.3, 59.4, 67.1 y un valor ruidoso en 0 (es imposible tener 0 en BMI).
* DiabetesPedigreeFunction [0.078, 1.182] además aparecen varios valores anómalos que no vamos a considerar como ruido porque en estudios médicos hay un pequeño porcentaje que se diferencia de los demás.
* Age [21, 64] con varios valores anómalos (el mayor de estos 81).

Además, estos diagramas nos brindan más información que puede resultar interesante para este problema:
* Pregnancies, SkinThickness, Insulin, DiabetesPedigreeFunction y Age suelen concentrar sus valores bajos cerca de la mediana y sus valores altos se expanden aún más ya que la mediana se encuentra más cerca de el valor minimo que del valor máximo del diagrama de cajas.
* Glucose, BloodPressure, BMI tienen diagramas de cajas más simétricos que los anteriores ya que como hemos visto en el histograma siguen una distribución normal


A continuación, vamos a ver qué cantidad de datos ruidosos tiene cada variable para estudiar qué variables presentan una cantidad de errores significante:

In [ ]:
df_ruidosos=utils.df_ruidosos(data_train)

In [ ]:
df_ruidosos.head(2)

Después de ver la cantidad de datos ruidosos y no ruidosos, nos damos cuenta de que Skin Thickness tiene más de un 20% de valores ruidosos, es una variable con demasiado ruido para tenerla en cuenta a la hora de realizar el estudio.

In [ ]:
data_train.describe(include="number")

In [ ]:
data_train.describe(include="category")

### Resumen

Después de analizar los datos sacamos las siguientes conclusiones:
* SkinThickness tiene demasiados valores ruidosos por lo que es una candidata a ser imputada en el siguiente paso.
* Insulin tiene varios valores anómalos que pueden ser importantes en este estudio médico.
* Encontramos distintas relaciones entre variables por lo que estas variable son importantes a la hora de predecir. Sabemos que esto es importante en los árboles de decisión.
* Los rangos que aparecen en el diagrama de cajas coinciden en su mayoría con los creados en los histogramas. Esto se tendrá en cuenta para suavizar ruido, imputar valores perdidos,...
* Viendo el diagrama de puntos y el histograma, este problema sería mejor discretizarlo usando kmeans porque en algunos casos existe una división entre valores notable, por lo que se podría discretizar usando clustering (en algunos casos se ve una separación entre dos grupos).
* En algunas todas las variables no merece la pena quitar los valores anómalos (esto lo vemos en el pairplot) ya que estos valores pueden ser muy importantes (puede ser que estos anómalos sean los que provoquen la diabetes, debemos de recordar que los casos con diabetes son menos).


## 1.3. Preprocesamiento de datos

Crearemos varios transformadores que se ejecutaran sobre los datos crudos y que serán añadidos en las pipelines correspondientes.

### Limpieza de datos

Los transformadores que aparecen a continuación se encargan de suavizar el ruido de nuestros datos. 

Vamos a crear un transformador que se encarga de poner los valores que no pertenecen a un rango a nulos. No eliminaremos valores anómalos ya que, como hemos dicho en el resumen del analisis exploratorio, en este caso los valores anómalos nos brindan bastante información.

In [ ]:
nan_put=utils.ManualNanPut({'Glucose':[50,250], 'BloodPressure':[10,150], 'Pregnancies':[0,50], 'SkinThickness':[1,80], 'Insulin':[0,650], 'BMI':[10,65], 'DiabetesPedigreeFunction':[0.0,3.0], 'Age':[0,100], })

Después de borrar todos los valores ruidosos, realizaremos una imputación de valores perdidos (preservando la media):

In [ ]:
imp = SimpleImputer(missing_values=float('nan'), strategy='mean')

### Reducción de datos

#### Imputación de variables

Después, como hemos visto en el analisis exploratorio de datos vamos a imputar la variable 'SkinThickness' ya que es una variable con demasiado ruido.

In [ ]:
feature_selector=utils.ManualFeatureSelector([0,1,2,4,5,6,7])

#### Discretización

Tal y como hemos visto en el análisis exploratorio de datos, lo mejor sería discretizar usando la estrategia de k-medias en 2 intervalos:

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="kmeans")

## 1.4. Creación de nuestros modelos y pipelines

Creamos el clasificador 0R:

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

Creamos el árbol de decisión:

In [ ]:
tree_model_classifier = DecisionTreeClassifier(random_state=seed)

### Pipeline

Creamos el pipeline del árbol de decisión con discretización:

In [ ]:
discretize_tree_model = make_pipeline(nan_put, imp, feature_selector, discretizer, tree_model_classifier)

Creamos el pipeline del árbol de decisión sin discretización

In [ ]:
tree_model = make_pipeline(nan_put, imp, feature_selector, tree_model_classifier)

NOTA: No es necesario crear el pipeline para 0R ya que en este modelo siempre se va a elegir la clase mayoritaria (no necesitamos un preprocesamiento de datos).

## 1.5. Evaluación de modelos

Ahora es el momento de entrenar y validar nuestros clasificadores. Para ello, vamos a usar una matriz de confusión, precisión y recall ya que en problemas médicos es lo más usual (ya que en este tipo de problemas la variable clase no se encuentra balanceada).

Precisión= TP/(TP+FP)
Recall= TP/(TP+FN)

También dibujaremos la curva ROC para analizar con mayor certeza los resultados.

### Clasificador 0R

In [ ]:
utils.evaluate(zero_r_model, X_train, X_test, y_train, y_test)

Precisión=0/(0+0)=0
Recall=0/(0+67)=0

Obviamente con 0R hemos obtenido malos resultados ya que 0R clasifica siempre como la clase mayoritaria. No tener diabetes siempre será la clase mayoritaria por lo que siempre se predice como no diabetes. Un modelo que siempre predice como no diabetes no es útil para este caso, se busca un modelo que sea capaz de encontrar los diabéticos.

También tenemos que el area de la curva ROC es 0.5, este valor nos aclara que 0R es un mal modelo ya que no tiene capacidad discriminatoria. Al ver que la curva ROC es la diagonal, concluimos que OR es un mal modelo en este caso.

### Árbol de decisión sin discretizar

In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

Concluimos que el 53% de los casos clasificados como diabéticos, lo son realmente, sabiendo además que el 66% de los diabéticos ha sido diagnosticado como diabéticos. Estos números no son muy altos, por lo que tenemos un modelo regular.

Además, vemos que el area debajo de la curva ROC es 0.67236, por lo que, como hemos dicho antes, es un modelo regular para análisis médico.


### Árbol de decisión discretizado

In [ ]:
utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

Concluimos que el 65% de los casos clasificados como diabéticos, lo son realmente, sabiendo además que el 51% de los diabéticos ha sido diagnosticado como diabéticos. En este caso hemos obtenido una mayor precision pero recall es algo peor que el anterior modelo. 

La curva ROC tiene un area de 0.68173, algo mejor que el anterior modelo, aún así, no es un buen modelo.

### Resumen de resultados

Habiendo estudiado los tres modelos anteriores, ninguno de ellos son buenos modelos, pero el mejor de ellos es el árbol de decisión discretizado.

# 2. Wisconsin database

## 2.1 Acceso y almacenamiento de datos

Cargamos la base de datos:

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"

data = utils.load_data(filepath, index, target)

Obtenemos los 5 primeros ejemplos para visualizar los datos.

In [ ]:
data.head(5)

De aquí podemos obtener la siguiente información:

* La mayoría de variables son valores continuos (todos los que aparecen son decimales).
* Tenemos 31 variables predictoras y 1 variable clase (diagnosis)
* La variable clase es discreta (valores M o B).
* Aparece una variable sin nombre con todos los valores NaN.

No aparecen todas las variables en la tabla por eso hemos hecho este pequeño estudio para ir familiarizándonos con los datos. Este estudio se llevará en profundidad en el apartado "Análisis exploratorio de datos"

In [ ]:
data.sample(5, random_state=seed)

Ahora procedemos a dividir los datos:

In [ ]:
(X, y) = utils.divide_dataset(data, target="diagnosis")

Volvemos a obtener los ejemplos aleatorios para comprobar que se ha realizado correctamente la división

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Dividimos los datos en train/test para tener una parte de los datos para entrenar y otra para llevar a cabo la evaluación de nuestro modelo. En nuestro caso hemos puesto que el 75% de los datos serán train y el 25% test:

In [ ]:
train_size = 0.75

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Obtenemos un subconjunto de cada:

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
X_test.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)

In [ ]:
y_test.sample(5, random_state=seed)

Unimos las variables predictoras con la variable clase en ambos conjuntos de datos:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

In [ ]:
data_test = utils.join_dataset(X_test, y_test)

In [ ]:
data_train.sample(5, random_state=seed)

In [ ]:
data_test.sample(5, random_state=seed)

## 2.2. Análisis exploratorio de datos

Antes de comenzar el preprocesamiento vamos a observar las características del conjunto de datos (de esta forma, haremos el preprocesamiento de una manera más efectiva).

### Descripción del conjunto de datos

In [ ]:
data_train.shape

Tal y como se puede observar, el conjunto de datos de entrenamiento está formado por 426 casos y 32 variables (31 variables predictoras y 1 variable clase).

In [ ]:
data_train.info(memory_usage=False)

Viendo esto, concluimos que la variable Unamed: 32 tiene todos los valores NaN (no aporta ningún valor al problema). No encontramos ningún valor perdido en ninguna de las otras variables (todas tienen 426 no nulos). Todas las variables predictoras son float. La variable clase es de tipo category.

Al analizar todas las variables, encontramos tres tipos de estas:
* El valor medio (el que vamos a usar para predecir).
* El error estandar (utilizaremos para encontrar valores ruidosos de las variables).
* Peores valores de cada variable (también se puede usar para predecir).


In [ ]:
y_train.cat.categories

Esto es, nuestra variable clase es multivariada con dos estados (B y M).

### Visualización de las variables

In [ ]:
utils.plot_histogram(data_train)

 Las variables predictoras de tipo mean y worst que presentan una distribución normal:
* radius_mean aunque es un poco asimétrica. No tiene ruido aunque tiene valores anómalos en 27 y 28.
* texture_mean con un valor anómalo en 39.
* perimeter_mean parece que sigue una distribución normal aunque se encuentra algo asimétrica (aparentemente no tiene ningún valor anómalo).
* smoothness_mean presenta un valor anómalo en 0.16.
* symmetry_mean aparentemente no presenta ningún valor anómalo ni ruidoso.
* fractal_dimension_mean presenta valores anómalos a partir de 0.09. No tiene valores ruidosos.
* radius_worst con un valor anómalo en 36
* texture_worst sin ningún valor anómalo ni ruidoso aparentemente.
* perimeter_worst igual que perimeter_mean presenta una distribución normal aunque se encuentra algo asimétrica. Tiene un valor anómalo en 250.
* smoothness_worst tiene dos valores anómalos en 0.215 y 0.22.
* symmetry_worst con bastantes valores anómalos a partir de 0.5.
* fractal_dimension_worst con valores anómalos a partir de 0.14.

A continuación hablaremos sobre las variables que no representan distribuciones normales:
* concave point_worst y concave point_mean: parecen una mixtura de distribuciones normales. Aparentemente no tienen valores anómalos ni ruidosos.
* area_mean: es parecida a una distribución normal pero no llega a serlo porque sus datos se encuentran balanceados hacia la izquierda. Tiene valores anómalos a partir de 2000.
* area_worst: tiene una distribución parecida a area_mean. Tiene un valor anómalo en 4200.
* compactness_mean y compactness_worst: también tiene una distribución balanceada hacia la izquierda. compactness_mean tiene valores anómalos a partir de 0.27 y compactness_worst a partir de 0.85.
* concavity_mean y concavity_worst: tienen una distribución como las anterior. concavity_mean tiene valores anómalos a partir de 0.4 y concavity_worst a partir de 1.
 
En las gráficas anteriores no se han encontrado valores ruidosos (no había valores que parecieran erróneos según el significado de la variable) por lo tanto, vamos a analizar las variables de tipo "se" (error estandar) para así poder encontrar los valores ruidosos de las variables:
* radius_se: hay nueve valores que se alejan de la distribución de errores (a partir de 1.2). Estos valores los consideraremos como ruidosos.
* texture_se: hay 14 valores ruidosos a partir de 2.5.
* perimeter_se: tiene dos valores ruidosos a partir de 18.5.
* area_se: tiene cuatro valores ruidosos a partir de 229.99.
* smoothness_se: hay nueve valores ruidosos a partir de 0.015.
* compactness_se: tiene 131 varios valores ruidosos a partir de 0.03
* concavity_se : tiene 216 valores ruidosos a partir de 0.025.
* concave points_se : tiene 105 valores ruidosos a partir de 0.015.
* simmetry_se : tiene 19 valores ruidosos a partir de 0.036.
* fractal_dimension_se : tiene 18 valores ruidosos a partir de 0.0085.

Para analizar la cantidad de valores ruidosos y el umbral por el que consideramos que un valor es ruidoso hemos comparado gráficas de "mean" y "se" para ver los valores promedios. Por ejemplo como concavity tiene valores muy pequeños, el mínimo error estandar ya ocasiona que sea un valor ruidoso


Continuamos visualizando las variables categóricas del problema:

In [ ]:
utils.plot_barplot(data_train)

A partir de esta gráfica concluimos que las dos clases de la variable objetivo del problema no tienen el mismo número de casos, entonces no es una muestra balanceada, ya que hay más ejemplos con un tumor benigno. Es algo que suele pasar en los estudios médicos, el porcentaje de la población afectada suele ser menor.

A continuación, vamos a proceder a realizar un análisis multivariado:

Para hacer nuestro análisis multivariado vamos analizar por un lado las variables tipos "mean" y por el otro "worst" para que se pueda analizar con más claridad.

In [ ]:
data_train_aux1=data_train[['radius_mean','texture_mean','perimeter_mean','area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'diagnosis']]

In [ ]:
utils.plot_pairplot(data_train_aux1, target="diagnosis")

Viendo este análisis multivariado, primero analizamos las variables predictoras por separado:
* En radius vemos que los puntos donde el tumor es Benigno se encuentran a la izquierda de nuestras gráficas. Es decir, los tumores malignos tienden a tener un radio mayor que los tumores benignos.
* En texture podemos ver que se cumple lo mismo que la variable predictora anterior, aunque aquí los puntos están mas dispersos y la diferencia no es tan notable ya que encontramos puntos azules a la derecha (hay tumores benignos con textura alta).
* En perimeter hemos visto que ocurre lo mismo que en las anteriores gráficas. De hecho, observamos que las gráficas de radius y perimeter son muy parecidas (tiene sentido porque si los tumores tienden a ser circulares el perimetro y el radio estarán relacionados directamente).
* En la variable predictora area ocurre lo mismo que en perimeter aunque, está algo más dispersa.
* En Smooothness las gráficas no está tan clara como las anteriores, pero, aún así, se puede ver como los tumores benignos tienden a tener valores de smoothness más bajos.
* En la variable predictora Compactness pasa lo mismo que en la anterior variable.
* En concavity vemos como los tumores benignos tienden a tener valores bastantes bajos (con algún valor que se aleja de la media).
* Concave points muestra unas gráficas algo más claras que las anteriores donde los puntos benignos están más a la izquierda que los malignos.
* Symmetry tiene valores bastantes dispersos (más que las anteriores variables) y no podemos sacar una conclusión clara donde podamos diferenciar los tumores benignos y malignos.
* En fractal_dimmension encontramos los tumores benignos con valores más altos que los malignos.

Ahora procedemos a analizar variables predictoras de forma conjunta:

* Las variables radius, perimeter y area estan muy relacionadas (directamente) entre ellas ya que sus gráficas conjuntas se muestran como una línea creciente y tienen gráficas muy parecidas entre ellas.
* Existe una relación entre concave points y perimeter ya que se puede ver una gráfica con puntos crecientes. Además, se diferencian los puntos de los valores malignos (concave points y perimeter más altos) a los valores benignos (concave points y perimeter más bajos). Obviamente, también existe una relación de concave points con radius y area ya que estas dos últimas variables predictoras son muy parecidas a perimeter.


Ahora procedemos a realizar el mismo análisis pero con la variable worst.

In [ ]:
data_train_aux2=data_train[['radius_worst','texture_worst','perimeter_worst','area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'diagnosis']]

In [ ]:
utils.plot_pairplot(data_train_aux2, target="diagnosis")

Aquí encontramos varias diferencias con el anterior estudio:
* En este caso Area se diferencia algo más de perimeter y radius.
* Texture tiene los valores de ámbas clases más dispersos por lo que la diferencia entre clases se disipa.
* En este caso los valores clase de las gráficas de concave points se diferencian más entre sí (los tumores benignos tienden a tener menor valor de concave points que los malignos).
* Fractal_dimension ha cambiado sus gráficas. En este caso la variable predictora es algo peor ya que la diferencia entre clases no se llega a ver como en el anterior.

A continuación, vamos a volver a hacer un análisis multivariado usando "mean" y luego "worst" con un heatmap

In [ ]:
utils.plot_heatmap(data_train_aux1)

En esta gráfica podemos encontrar varias relaciones claras:

* Radius, perimeter y area tienen una relación directa como hemos podido comprobar antes.
* Concave poinst con radius perimeter y area tiene una relación directa (esto también lo hemos podido comprobar en la anterior gráfica).
* Compactness concavity y concave points también están relacionadas diréctamente.

In [ ]:
utils.plot_box(data_train)

Con estos diagramas de cajas podemos sacar rangos más precisos sobre cada variable (sin tener en cuenta los valores anómalos representados por puntos en el diagrama). Obtendremos intervalos mas exáctos que en el histograma (ya que en este tipo de gráficos aparecen valores máximos, mínimos y anómalos). Empezaremos con las variables predictoras de tipo mean:

* Radius: [6.981, 22.27] con ocho valores anómalos desde 23.09 hasta 28.11.
* Texture: [10.38, 29.43] con siete valores anómalos desde 29.81 hasta 39.28.
* Perimeter: [43.79, 153.5] con seis valores anómalos desde 155.1 hasta 188.5.
* Area: [143.5, 1386] con trece valores anómalos desde 1404 hasta 2501.
* Smoothness: [0.06251, 0.1326] con cinco valores anómalos desde 0.1371 hasta 0.1634.
* Compactness: [0.01938, 0.2276] con diez valores anómalos desde 0.2293 hasta 0.3454.
* Concavity: [0, 0.2871] con diez valores anómalos desde 0.3001 hasta 0.4268.
* Concave points: [0, 0.152] con siete valores anómalos desde 0.1562 hasta 0.2212.
* Symmetry: [0.1203, 0.2459] con un valor anómlao en 0.106 y once anómalos desde 0.2495 hasta 0.304.
* Fractal_dimension: [0.04996, 0.07871] con doce valores anómalos desde 0.0795 hasta 0.09744.

Continuamos con las variables de tipo worst:
* Radius: [7.93, 28.4] con seis valores anómalos desde 29.17 hasta 36.04.
* Texture: [12.49, 40.68] con cinco valores anómalos desde 41.61 hasta 47.16.
* Perimeter: [50.41, 188.5] con once valores anómalos desde 195 hasta 251.2.
* Area: [185.2, 2089] con veinte valores anómalos desde 2145 hasta 4254.
* Smoothness: [0.08125, 0.1862] con un valor anómalo en 0.07117 y siete valores anómalos desde 0.1878 hasta 0.2226.
* Compactness: [0.02729, 0.6247] con trece valores anómalos desde 0.659 hasta 1.058.
* Concavity: [0, 0.7681] con nueve valores anómalos desde 0.8216 hasta 1.252.
* Concave points: [0, 0.291] sin valores anómalos.
* Symmetry: [0.1566, 0.4154] con 17 valores anómalos desde 0.4228 hasta 0.6638.
* Fractal_dimension: [0.05521, 0.1224] con 14 valores anómalos desde 0.1233 hasta 0.2075.


Además, estos diagramas nos brindan más información que puede resultar interesante para este problema:

* Los diagramas de caja de las variables que siguen distribuciones normales son simétricas.
* Además, los valores anómalos casi siempre se encuentran con valores mayores del rango.

A continuación vamos a analizar la anchura de los intervalos de confianza del 95% (fórmula intervalo confianza media+-z\*se). Compararemos las variables de tipo 'mean' con la anchura del intervalo de confianza.

In [ ]:

variables=['radius','texture','perimeter','area', 'smoothness', 'compactness', 'concavity', 'concave points', 'symmetry', 'fractal_dimension']
data_train_anchura=utils.anchura_intervalo_confianza(variables, data_train)
for v in variables:
    data_train_anchura[v+"_mean"]=data_train_aux1[v+"_mean"]
utils.plot_histogram(data_train_anchura)

Combinando diagramas de variables de tipo mean y anchura vemos intervalos demasiado anchos en compactness, concavity y concave points.

También encontramos variables con intervalos anchos (no tanto como los otros), por ejemplo fractal_dimension, symmetry y smoothness.

Las variables que mejores intervalos de confianza (del 95%) nos brindan son: radius, texture, perimeter y area.

Por lo que debemos de tener en cuenta que las variables más "fiables" son radius,texture, perimeter y area. Luego encotramos variables que no son tan buenas como fractal_dimension, symetry y smoothness. Por último, las variables compactness, concavity y concave son variables poco fiables ya que tienen un intervalo de confianza (medio) demasiado ancho.

### Resumen

Después de analizar los datos sacamos las siguientes conclusiones:
* Tenemos variables muy parecidas en 'mean': radius, perimeter y area
* En 'worst' la variable area es algo más distinta de radius y perimeter.
* Los tumores benignos suelen casi todos los valores de las variables predictoras más bajos (menos en fractal_dimension).
* Las varibles que más confianza brindan (en cuanto a error estandar) son radius, texture, perimeter y area.
* Las variables que menos confianza brindan son compactness, concavity y concave por su gran error estandar (deben de ser imputadas).
* Usaremos los rangos de los diagramas de cajas para suavizar los valores anómalos.
* En el pairplot podemos ver que si dividimos los rangos en 5 hay partes donde encontraremos solo puntos azules en la izquierda y puntos rojos en la derecha de algunos diagramas. Por ello, pensamos que una discretización uniforme en 5 beans puede ir bien para este problema.



## 2.3. Preprocesamiento de datos

Crearemos varios transformadores que se ejecutaran sobre los datos crudos y que serán añadidos en las pipelines correspondientes.

### Limpieza de datos

Los transformadores que aparecen a continuación se encargan de suavizar el ruido de nuestros datos. 

Primero creamos un transformador que se encarga de poner como valores nulos los valores anómalos de cada variable:

In [ ]:
nan_put_anomalos=utils.ManualNanPut({'radius_mean':[6.981,22.27], 'texture_mean':[10.38,29.43], 'perimeter_mean':[43.79,153.5], 'area_mean':[143.5,1386], 'smoothness_mean':[0.06251, 0.1326], 'compactness_mean':[0.01938, 0.2276], 'concavity_mean':[0,0.2871], 'concave points_mean':[0,0.152],'symmetry_mean':[0.1203, 0.2459], 'fractal_dimension_mean':[0.04996, 0.0787], 'radius_worst':[7.93,28.4], 'texture_worst':[12.49,40.68], 'perimeter_worst':[50.41,188.5], 'area_worst':[185.2,2089], 'smoothness_worst':[0.08125, 0.1862], 'compactness_worst':[0.02729, 0.6247], 'concavity_worst':[0,0.7681], 'concave points_worst':[0,0.291],'symmetry_worst':[0.1506, 0.4154], 'fractal_dimension_worst':[0.05521, 0.1224] })

Primero creamos un transformador que se encarga de poner como valores nulos los valores ruidosos de cada variable:

In [ ]:
nan_put_ruidosos=utils.ManualNanPutSE({'radius':1.2, 'texture':2.5, 'perimeter':18.5, 'area':229.99, 'smoothness':0.015, 'compactness':0.03, 'concavity':0.025, 'concave points':0.015,'symmetry':0.036, 'fractal_dimension':0.0085 })

Después de borrar todos los valores ruidosos y anómalos, realizaremos una imputación de valores perdidos (preservando la media):

In [ ]:
imp = SimpleImputer(missing_values=float('nan'), strategy='mean')

### Reducción de datos

#### Imputación de variables

Después, como hemos visto en el analisis exploratorio de datos vamos a imputar las variables 'concavity', 'compactness' y 'concavity points' ya que son variables con demasiado ruido. También, en la parte de 'mean' quitaremos perimeter y area porque son muy parecidas a radius (es decir, estan muy relacionadas). En la parte de worst solo quitaremos perimeter ya que area se diferencia un poco más de las otras dos. Además, quitaremos las variables de tipo 'se' ya que no se deben de usar para predecir.

In [ ]:
feature_selector=utils.ManualFeatureSelector([0, 1, 4, 8, 9, 20, 21, 23, 24, 28, 29])

#### Discretización

Tal y como hemos visto en el análisis exploratorio de datos, lo mejor sería discretizar usando la estrategia de k-medias en 2 intervalos:

In [ ]:
discretizer = KBinsDiscretizer(n_bins=5, strategy="uniform")

## 2.4. Creación de nuestros modelos y pipelines

Creamos el clasificador 0R:

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

Creamos el árbol de decisión:

In [ ]:
tree_model_classifier = DecisionTreeClassifier(random_state=seed)

### Pipeline

Creamos el pipeline del árbol de decisión con discretización:

In [ ]:
discretize_tree_model = make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, tree_model_classifier)

Creamos el pipeline del árbol de decisión sin discretización

In [ ]:
tree_model = make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, tree_model_classifier)

NOTA: No es necesario crear el pipeline para 0R ya que en este modelo siempre se va a elegir la clase mayoritaria (no necesitamos un preprocesamiento de datos).

## 2.5. Evaluación de modelos

Ahora es el momento de entrenar y validar nuestros clasificadores. Para ello, vamos a usar una matriz de confusión, precisión y recall ya que en problemas médicos es lo más usual (ya que en este tipo de problemas la variable clase no se encuentra balanceada).

Precisión= TP/(TP+FP) Recall= TP/(TP+FN)

También dibujaremos la curva ROC para analizar con mayor certeza los resultados.

### Clasificador 0R

In [ ]:
utils.evaluate2(zero_r_model, X_train, X_test, y_train, y_test, 'M')

Precisión=0/(0+0)=0 Recall=0/(0+53)=0

Obviamente con 0R hemos obtenido malos resultados ya que 0R clasifica siempre como la clase mayoritaria. El tumor benigno siempre será la clase mayoritaria por lo que siempre se predice como benigno. Un modelo que siempre predice como benigno no es útil para este caso ya que nunca nos ayudará a encontrar los tumores malignos (y diagnosticarlos correctamente).

### Árbol de decisión sin discretizar

In [ ]:
utils.evaluate2(tree_model,
               X_train, X_test,
               y_train, y_test, 'M')

Concluimos que el 87% de los casos clasificados como malignos, lo son realmente, sabiendo además que el 88% de los tumores malignos ha sido diagnosticado como tal. Tenemos un buen comportamiento en este caso ya que la precision y recall son altos.

### Árbol de decisión discretizado

In [ ]:
utils.evaluate2(discretize_tree_model,
               X_train, X_test,
               y_train, y_test, 'M')

Concluimos que el 90% de los casos clasificados como malignos, lo son realmente, sabiendo además que el 90% de los tumores malignos ha sido diagnosticado como tal. Tenemos un muy buen comportamiento en la precisión y en el recall.

### Resumen de resultados

Habiendo estudiado los tres modelos anteriores 0R es un mal modelo (no es conveniente llevarlo a la práctica). Los dos modelos siguientes se comportan mucho mejor siendo el discretizado el que mejor se comporta de los tres.